<a href="https://colab.research.google.com/github/SzuHannah/ML-with-LCI/blob/main/PerhapsEfficientMultiTask.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import math
import numpy as np
from numpy import linalg as LA

#### Implementation of Liu and Ye (2009)
- reference  
  - [Liu and Ye(2009) paper](https://arxiv.org/pdf/1205.2631.pdf)
  - [Original MatLab code](https://github.com/jiayuzhou/SLEP/blob/master/SLEP/functions/L1Lq/Lq1R/mcLeastR.m)  
  - [Python code](https://github.com/jundongl/scikit-feature/blob/master/skfeature/function/sparse_learning_based/ls_l21.py)

In [133]:
# utility functions
def euclidean_projection (V, n_features, n_tasks, z, gamma):
  """
  L2-norm Regularized Euclidean Projection:
      min 1/2 ||W-V||_2^2 + rho * ||W||_2
  which has the closed form solution:
      W = {max(||V||_2 - rho , 0) / ||V||_2} * V
  where rho = z/gamma
  """
  W_projection = np.zeros((n_features, n_tasks))
  for i in range(n_features):
    rho = z/gamma
    if LA.norm(V[i, :]) > rho: # i.e. when max(||V||_2 - rho , 0) = ||V||_2 - rho
      W_projection[i, :] = (1 - rho/LA.norm(V[i, :])) * V[i, :]
    else:
      W_projection[i, :] = np.zeros(n_tasks)
  return W_projection

def calculate_l21_norm(X):
  """
  L21-norm of a matrix X:
    \sum ||X[i,:]||_2
  """
  l21_norm = np.sqrt(np.multiply(X, X).sum(1)).sum()
  return l21_norm

def init_factor(W_norm, XW, Y, z):
  """
  Iinitialize the starting point of W, according to the Liu and Ye (2009)'s code
  """
  n_samples, n_tasks = XW.shape
  # numerator = (WX')*Y - z*(W_norm)
  a = np.inner(np.reshape(XW, n_samples*n_tasks), np.reshape(Y, n_samples*n_tasks)) - z * W_norm
  # denominator = (Frobenius norm of XW) ** 2
  b = LA.norm(XW, 'fro') ** 2
  ratio = a/b
  return ratio


## for testing
def construct_label_matrix_pan(label):
    """
    This function converts a 1d numpy array to a 2d array, for each instance, the class label is 1 or -1
    """
    n_samples = label.shape[0]
    unique_label = np.unique(label)
    n_classes = unique_label.shape[0]
    label_matrix = np.zeros((n_samples, n_classes))
    for i in range(n_classes):
        label_matrix[label == unique_label[i], i] = 1
    label_matrix[label_matrix == 0] = -1

    return label_matrix.astype(int)

def feature_ranking(W):
    """
    This function ranks features according to the feature weights matrix W
    """
    T = (W*W).sum(1)
    idx = np.argsort(T, 0)
    return idx[::-1]

def nonzero_feature(W):
    """
    This function non-zero features of the weights matrix W
    """
    T = (W*W).sum(1)
    idx = np.argwhere(T!=0)
    return idx.flatten()

In [153]:
# main functions
def init_startingPoint(X, Y, z):
  XtY = X.T @ Y # compute X'Y
  W = XtY # initialize a starting point
  XW = X @ W  # compute XW = X*W
  W_norm = calculate_l21_norm(W) #compute l2,1 norm of W
  if W_norm >= 1e-6:
    ratio = init_factor(W_norm, XW, Y, z)
    W = ratio * W
    XW = ratio * XW
  return XtY, W, XW, W_norm 

def l21_proximal(X, Y, z, **kwargs):
  """
  X: input data matrix, shape = (n_samples, n_features)
  Y: input response matrix, shape = (n_samples, n_tasks)
  z: regularization parameter
  kwargs: 
    verbose: True if user want to print out the objective function value in each iteration, false if not
  """
  # process verbose:
  if 'verbose' not in kwargs:
    verbose = False
  else:
    verbose = kwargs['verbose']

  #shape of X, Y
  n_samples, n_features = X.shape
  n_samples, n_tasks = Y.shape

  # initialization
  XtY, W, XW, W_norm = init_startingPoint(X,Y,z)

  #--- Armijo Goldstein line search + accelerated gradient descent (proximal gradient descent) ---
  # initialize step size gamma = 1
  gamma = 1 #gamma=L in the original code

  #Initialize Wp (W in previous step) to W; XWp to XW; tp (t in previous step) to 0; t to 1
  XWp = XW
  WWp = np.zeros((n_features, n_tasks))
  tp = 0 #at initialization, this is t_{-1} in the paper, where t_{i} is the upper bound for each ||W(i)||, i.e. ||W(i)|| <= t_{i}
  t = 1 #at initialization, this is t_{0} in the paper

  #whether converged or not
  no_improvement = False
  
  max_iter = 1000
  gammas = np.zeros(max_iter) #collection of the step size for each iteration
  objs = np.zeros(max_iter) #collection of objective functions at each iteration

  for iter in range(max_iter):
    # step 1: compute search point S based on Wp and W' define alpha to be the decreasing factor for line search
    alpha = (tp - 1)/t
    S = W + alpha*WWp

    # step2: linear search for gamma and compute the new apporximation slution W
    XS = XW + alpha*(XW - XWp) 
    XtXS = X.T @ XS # compute X'*XS
    GatS = XtXS - XtY # compute gradient G at point S: G = X'(XS-Y) for later use to find the new approximation solution
    ## copy W and XW to Wp and XWp
    Wp = W
    XWp = XW

    while True:
      ## let S walk in a step in the antigradient of S to get V; then, do L1/L2-norm regularized euclidean projection for V
      ## in the paper, it was new_apporx_W = projection(S - 1/gamma * G), where G is gradient at point S
      V = S - (1/gamma) * GatS ### ie. V = "gradient descent step" of S 
      W = euclidean_projection(V, n_features, n_tasks, z, gamma) ### new_approx_W 
      ## difference between the new approximate solution W and the search point S
      V = W - S
      ## update XW and calculate XV
      XW = X @ W
      XV = XW - XS
      ## define r_sum = sqared(Frobinius norm of V); l_sum = squared(Frobinius norm of XV)
      r_sum = LA.norm(V, 'fro')**2
      l_sum = LA.norm(XV, 'fro')**2
      ## check if the "gradient descent step" makes litte improvement
      if r_sum <= 1e-20:
        no_improvement = True
        break
      ## the condition is ||XV||_2^2 <= gamma * ||V||_2^2
      if l_sum <= gamma * r_sum:
        break
      else:
        gamma = max(2*gamma, l_sum/r_sum)
    gammas[iter] = gamma

    # step 3: update tp and t, and check if converged
    tp = t
    t = (1+math.sqrt(1 + 4*t**2)) / 2

    WWp = W - Wp
    XWY = XW - Y 

    ## calculate obj
    objs[iter] = LA.norm(XWY, 'fro') ** 2 / 2
    objs[iter] += z * calculate_l21_norm(W)

    if verbose:
      print('obj function at iter {0}: {1}'.format(iter+1, objs[iter]))
    
    if no_improvement is True:
      break

    ## check convergence
    if iter >= 1 and math.fabs(objs[iter] - objs[iter - 1]) < 1e-3:
      break
  return W, objs, gammas



#### Test on LCI data

In [ ]:
import pandas as pd
X = pd.read_csv("X_data.csv")
Y = pd.read_csv("Y_data.csv", usecols = ["climate change","fossil depletion","metal depletion", "human toxicity"])

In [ ]:
X['size'],sizetype=pd.factorize(X['size'])
X['country'],countrytype = pd.factorize(X['country'])
X['powertrain'],powertrain = pd.factorize(X['powertrain'])

In [ ]:
numericcols = list(X.select_dtypes('float64').columns)
numericcols.append("year")
catcols = list(X.select_dtypes('int64').columns)
catcols.remove("year")
cols = X.columns
len(numericcols), len(catcols), len(cols)

(77, 3, 80)

In [ ]:
X # country has 7 levels, size has 9 levels, powertrain has 9 levels

In [ ]:
# comparison test on LCI data
## normalize data as suggested by Liu and Ye (2009)'s paper

### normalize data to make life easier
from sklearn.preprocessing import Normalizer

normX = Normalizer().fit_transform(X) 
normY = Normalizer().fit_transform(Y)

## elastic net from sklearn normalize data by default:
## ref: https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.ElasticNet.html


In [156]:
from sklearn import linear_model

def main(X, Y, colnames):
  n_samples, n_features = X.shape
  # split data into 5 folds
  ss = KFold(n_splits = 5)
  reg = linear_model.LinearRegression()

  # add up the mse from each fold
  total_mse = 0
  for train, test in ss.split(X):
    W, objs, gammas = l21_proximal(X[train], Y[train], 0.05, verbose = False)
    # obtain features with weight > 0
    idx = nonzero_feature(W)
    selected_features = X[:, idx]
    selected_features_colname = colnames[idx]
    # use the features to train a linear model (or can use other model to train)
    #reg.fit(selected_features[train], Y[train])
    #Y_predict = reg.predict(selected_features[test])
    Y_predict = X[test] @ W
    # obtain mse from the model
    mse = mean_squared_error(Y[test], Y_predict)
    total_mse = total_mse + mse
  average_mse = float(total_mse)/5
  #examine W from the last iter
  return selected_features_colname, average_mse 

if __name__ == '__main__':
  critical_feats, average_mse = main(normX, normY, X.columns) 
  print('MSE: ', average_mse)
  print(critical_feats)


MSE:  0.04939960199422524
Index(['LHV fuel MJ per kg', 'auxilliary power base demand',
       'average passenger mass', 'battery cell production energy',
       'battery lifetime kilometers',
       'battery onboard charging infrastructure cost', 'cargo mass',
       'combustion exhaust treatment cost', 'combustion fixed mass',
       'combustion powertrain cost per kW', 'cooling thermal demand',
       'electric fixed mass', 'electric powertrain cost per kW',
       'energy battery cost per kWh', 'energy battery mass',
       'fuel cell cost per kW', 'fuel cell lifetime hours',
       'fuel cell power area density', 'fuel mass', 'fuel tank cost per kg',
       'glider base mass', 'glider cost intercept', 'heat pump cost',
       'heating thermal demand', 'inverter mass', 'kilometers per year',
       'lifetime kilometers', 'power battery cost per kW', 'year'],
      dtype='object')


#### Benchmark

In [154]:
# timing comparison
from sklearn.linear_model import MultiTaskElasticNet
import time
from statistics import mean
def main(X, Y, colnames):
  n_samples, n_features = X.shape
  # split data into 5 folds
  ss = KFold(n_splits = 5)

  # l1,2 with proximal gradient
  prox_start = time.time()
  for train, test in ss.split(X):
    W, objs, gammas = l21_proximal(X[train], Y[train], 0.05, verbose = False) # max_iter parameter within the function makes a difference in time cost
    # obtain features with weight > 0
    idx = nonzero_feature(W)
    selected_features = X[:, idx]
    selected_features_colname = colnames[idx]
    # use the weights to build a linear model
    Y_predict_pr = X[test] @ W
  
  prox_timeCost = time.time()-prox_start
  print("proximal gradient average time: %.3f"% prox_timeCost)
if __name__ == '__main__':
  main(normX, normY, X.columns)

proximal gradient average time: 35.527


In [155]:
from numpy import std
from sklearn.model_selection import cross_val_score, RepeatedKFold
#1 time 5-fold
start = time.time()
model = MultiTaskElasticNet(alpha = 0.05, l1_ratio = 0.05)
cv = RepeatedKFold(n_splits = 5, n_repeats = 1, random_state=1)
scores = cross_val_score(model, X, Y, scoring = 'neg_mean_squared_error', cv = cv, n_jobs = -1) #taking really long.. maybe change coordinate descent to proximal descent
print('MSE: %.3f(%.3f)'%(mean(scores), std(scores)))
model.fit(X,Y)
duration = time.time() - start
print("coordinate gradient took %s seconds"  % duration) # maybe because coordinate descent can take advante of the sparsity of the weight vector, so it's faster
                                                        # similarly saw in this: https://github.com/ngmarchant/prox_elasticnet/blob/master/demo/benchmark.ipynb

MSE: -0.002(0.000)
coordinate gradient took 20.984787225723267 seconds


#### Appendix

In [129]:
# test using the example in the original repo (classification data) to ensure code works
import scipy.io
from sklearn import svm
from sklearn.model_selection import cross_validate, KFold
from sklearn.metrics import accuracy_score, mean_squared_error

In [157]:
def main():
    # load data
    mat = scipy.io.loadmat('COIL20.mat')
    X = mat['X']    # data
    X = X.astype(float)
    y = mat['Y']    # label
    y = y[:, 0]
    Y = construct_label_matrix_pan(y)
    n_samples, n_features = X.shape    # number of samples and number of features

    # split data into 10 folds
    ss = KFold(n_splits=10, shuffle=True)

    # perform evaluation on classification task
    num_fea = 100    # number of selected features
    clf = svm.LinearSVC()    # linear SVM

    correct = 0
    for train, test in ss.split(X):
        # obtain the feature weight matrix
        Weight, obj, value_gamma = l21_proximal(X[train], Y[train], 0.1, verbose=False)

        # sort the feature scores in an ascending order according to the feature scores
        idx = feature_ranking(Weight)

        # obtain the dataset on the selected features
        selected_features = X[:, idx[0:num_fea]]

        # train a classification model with the selected features on the training dataset
        clf.fit(selected_features[train], y[train])

        # predict the class labels of test data
        y_predict = clf.predict(selected_features[test])

        # obtain the classification accuracy on the test data
        acc = accuracy_score(y[test], y_predict)
        correct = correct + acc

    # output the average classification accuracy over all 10 folds
    print('Accuracy:', float(correct)/10)

if __name__ == '__main__':
    main()

Accuracy: 0.975
